# 프로젝트 개요
- 강의명 : 2022년 K-디지털 직업훈련(Training) 사업 - AI데이터플랫폼을 활용한 빅데이터 분석전문가 과정
- 교과목명 : 빅데이터 분석 및 시각화, AI개발 기초, 인공지능 프로그래밍
- 프로젝트 주제 : Spaceship Titanic 데이터를 활용한 탑승유무 분류모형 개발 
- 프로젝트 마감일 : 2022년 4월 12일 화요일
- 강사명 : evan
- 수강생명 : 김민성

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 1. 라이브러리 및 데이터 불러오기
- 본 프로젝트 수행을 위한 필수 라이브러리를 불러온다. 

In [ ]:
# 데이터 가공 
import numpy as np 
import pandas as pd 

print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)

# 데이터 시각화
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns 
print("matplotlib version:", matplotlib.__version__)
print("seaborn version:", sns.__version__)

import matplotlib.ticker as mtick
plt.rcParams["figure.figsize"] = 10, 6
plt.rc("axes.spines", top=False, right=False)
palette = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
sns.set_palette(palette)

# train data
- 훈련 데이터를 살펴본다.
- 훈련 데이터에는 14개의 열이 있고, 8,693개의 행(샘플)이 있다.
- 훈련 데이터에는 119,378개의 특성이 있다.
- 훈련 데이터안에는 2,324개의 결측치가 있다.
- Transported 열은 훈련 데이터 안에서만 사용할 수 있다.
- 14개의 열 중에서 CryoSleep 열 안에 217개로 가장 많은 결측치가 있다.

In [ ]:
train_data = pd.read_csv("../input/spaceship-titanic/train.csv")
test_data = pd.read_csv("../input/spaceship-titanic/test.csv")
sample_submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")

train_data.shape, test_data.shape, sample_submission.shape

- train_data에 대해 살펴본다. 
- 먼저, 결측치가 있는 코드들이 있지만, 전체적으로 많은 편은 아님. 
- bool(1), float64(6), object(7)개로 구성이 되어 있음. 

In [ ]:
train_data.info()

- 테스트 데이터도 마찬가지로 비슷한 양상을 나타냄. 

In [ ]:
test_data.info()

# Column 설명
- PassengerId는 "AAAA_BB"형태이다. AAAA는 그룹을 나타내고, BB는 그룹안에서의 개개인을 지칭한다.
-  HomePlanet은 고향별을 나타낸다.
- CryoSleep은 여행중에 냉동수면 여부를 나타낸다.
- cabin은 승객이 머무르는 공간을 나타낸다. deck/number/side 형태로 구성된다. side의 P는 Port를 나타내고, S는 Starboard를 나타낸다.
- Age는 승객의 나이이다.
- VIP는 승객이 special VIP service 비용을 지불했는지를 보여준다.
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck는 각각 우주선의 편의시설에 승객이 비용을 지불했는지를 나타낸다.
- Name은 승객의 이름으로, first name, last name 순이다.
- Transported 칼럼이 타깃 데이터이다. 승객이 다른 차원으로 운송되었는지 여부를 보여준다. 훈련 데이터에는 Transported 칼럼이 있고, 테스트 데이터에는 Transported 칼럼이 있다. Transported 칼럼을 예측해야 할 것이다.

In [ ]:
sample_submission.info()

- 대회주제
    + Transported Column 설명을 보면, 다른 차원으로 이동하기 위한, 각 승객의 탑승을 했는지 못했는지를 예측하는 문제이며, 평가지표는 정확도로 판정함.
- 분류 정확도(Classification Accuracy)의 설명은 [Classification Accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy)에서 확인할 수 있다. 

# Step 2. 탐색적 자료 분석 (EDA)
- 데이터 시각화
- 산점도, 막대 그래프 등
- 그래프 해석해서 설명을 달아야 함
- 약간의 데이터 전처리

## 데이터 시각화를 위한 주요 함수 작성
- 훈련데이터의 구조는 다음과 같다. 
    + dtypes: bool(1), float64(6), object(7)
- 우선, 종속변수인 bool의 갯수에 대해 확인한다. 
- 갯수로 파악할 때는 비율은 큰 차이가 없는 것으로 확인했다. 즉, 데이터는 매우 균등하게 구성이 된 것이다. 

In [ ]:
# Transported value_counts 확인

train_data['Transported'].value_counts()

In [ ]:
train_data['CryoSleep'].value_counts()

- 그래프를 위한 사용자 지정함수를 만든다

In [ ]:
def cnt_bar(data, col_name):
    df = data[col_name].value_counts()
    fig, ax = plt.subplots(figsize=(10, 8))
    labels = [str(item) for item in list(data[col_name].value_counts().index)]
    bars = sns.countplot(x=col_name, data=data, color='lightgray', alpha=0.85, zorder=2, ax=ax)
    
    for bar in bars.patches:
        fontweight = "normal"
        color = "k"
        height = np.round(bar.get_height(), 2)
        if bar.get_height() == data[col_name].value_counts().values[0]:
            fontweight="bold"
            color="blue"
            bar.set_facecolor(color)
        ax.text(bar.get_x() + bar.get_width()/2., height+100, height+1, ha = 'center', size=12, fontweight=fontweight, color=color)
    ax.set_title(f'Bar Graph of {col_name}', size = 16)
    ax.set_xlabel(col_name, size = 16)
    ax.set_ylabel("No. Passengers", size = 16)

    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_position(("outward", 20))
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.grid(axis="y", which="major", color="lightgray")
    ax.grid(axis="y", which="minor", ls=":")
    
    plt.show()

- 이제 함수를 적용해본다. 
    + Cabin이나, PassengerID 등에는 적용할 수 없다. 

In [ ]:
cnt_bar(train_data, "Destination")

In [ ]:
cnt_bar(train_data, "HomePlanet")

In [ ]:
cnt_bar(train_data, "CryoSleep")

In [ ]:
cnt_bar(train_data, "VIP")

In [ ]:
cnt_bar(train_data, "Transported")

- 이번에는 Describe() 함수를 적용해본다. (데이터 요약)

In [ ]:
train_data.describe()

In [ ]:
# 칼럼명 확인
numeric_features = train_data.select_dtypes(include=[np.number])
numeric_features.columns

- 데이터 시각화

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(16, 10)) # 그래프의 행과 열 지정 및, 이미지 사이즈 지정
data = train_data.copy()
# data[numeric_features.columns].columns[0:]
for i, col in enumerate(data[numeric_features.columns].columns[0:]): # 좌표 평면 지정
    # print(i, col)
    if i <= 2:
        sns.boxplot(x=data["Transported"], y=data[col], ax=ax[0,i]) # 1행 좌표 평면
    else: 
        sns.boxplot(x=data["Transported"], y=data[col], ax=ax[1,i-4]) # 5행 좌표 평면
fig.suptitle('My Box Plots')
fig.tight_layout()
fig.subplots_adjust(top=0.95)

- 위 시각화 결과물을 놓고 보면, RoomService, FoodCount, Spa, VRDeck, ShoppingMall은 수치가 아닌 것처럼 보임. 
- 각셀의 value_counts 값 확인

In [ ]:
train_data['RoomService'].value_counts()

In [ ]:
train_data['FoodCourt'].value_counts()

In [ ]:
train_data['Spa'].value_counts()

- 실제 컬럼명에서 말하는 것도 다음과 같음. 
    + RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
- 즉, 위 데이터는 연속형 데이터라고 보기에는 비연속형 수치 데이터인 것을 확인함. 

# Step 3. 데이터 전처리
- Feature Engineering
- ML 모형을 돌리기 위해 표준화 등 / 원핫-인코딩
- 파생변수 (도출 변수)
    + 왜 이 변수를 만들었는지에 대한 여러분들의 설명 필요

## 불필요한 변수 제거 
- 모형에 쓰지 않을 변수들을 제거한다. 
    + PassengerId, Name
- Name은 모형을 개선할 때, 가족 유무 등 판단 시, 재활용할 가치는 있다.
- Cabin 역시 모형 개선 시, 문자열 처리를 통해 추가 변수를 도출 할 수는 있지만, 일단은 제거한다. 

In [ ]:
train_data['Name'].value_counts()

In [ ]:
train_data['Cabin'].value_counts()

In [ ]:
# 필요없는 열 제거(drop 함수 이용)
remove_cols = ['PassengerId', 'Name', 'Cabin']
PassengerId = test_data['PassengerId']

print("Before:", train_data.shape, test_data.shape)
train_data = train_data.drop(remove_cols, axis=1)
test_data = test_data.drop(remove_cols, axis=1)

print("After:", train_data.shape, test_data.shape)

- drop 함수를 이용하여 필요없는 행을 제거하였다.

## 결측치 
- 결측치 데이터를 추가하도록 한다. 결측치를 추가하기 위해 SimpleImputer 클래스를 사용하였다. 
- train_data의 결측치 패턴을 파악한 후, train_data 및 test_data 결과에도 적용한다. 
    + 주의 : 반드시 이렇게 해야 한다. (Data Leakage 방지)

- train_data , test_data에 있는 결측치들을 확인한 후 결측치 처리를 진행한다.

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

 - 결측치 처리


In [ ]:
from sklearn.impute import SimpleImputer

imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
STRATEGY = 'median'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train_data[imputer_cols])
train_data[imputer_cols] = imputer.transform(train_data[imputer_cols])
test_data[imputer_cols] = imputer.transform(test_data[imputer_cols])

print("train_data:\n", train_data.isnull().sum())
print("---")
print("test_data:\n", test_data.isnull().sum())

In [ ]:
imputer_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]
STRATEGY = 'most_frequent'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train_data[imputer_cols])
train_data[imputer_cols] = imputer.transform(train_data[imputer_cols])
test_data[imputer_cols] = imputer.transform(test_data[imputer_cols])

print("train_data:\n", train_data.isnull().sum())
print("---")
print("test_data:\n", test_data.isnull().sum())

In [ ]:
train_data.isnull().sum()

- 결측치가 사라진 것을 확인할 수 있다. 

## Categorical Feature Encoding
- 머신러닝 알고리즘은 수식으로 구성이 되어 있기 때문에 문자열의 경우 인코딩으로 변환을 주어야 한다. 
- 크게 두가지 방법이 존재한다. 
    + Ordinal Encoding 
        - 점수 : 0.78770
    + Onehot Encoding
        - 점수 : 0.78840
    + pd.get_dummies
        - 점수 : 0.78840
        
- 크게 두가지 결론을 얻을 수 있었음
    + 첫째, 본 데이터에서는 OneHotEncoding 방식이 Ordinal Encoding 보다 좋았음. 
    + 둘째, Onehot Encoding 방식과 pd.get_dummies 방식 차이는 없었음

In [ ]:
train_data.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# Target 데이터는 1과 0으로 바꿈
train_data['Transported'] = train_data['Transported'].map({True: 1, False: 0})
categorical_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

transformer = make_column_transformer(
    (OneHotEncoder(), categorical_cols),
    remainder='passthrough')

train_transformed = transformer.fit_transform(train_data[categorical_cols])
train_transformed_df = pd.DataFrame(train_transformed, columns=transformer.get_feature_names_out())
train_data = pd.concat([train_data, train_transformed_df], axis = 1)
train_data = train_data.drop(categorical_cols, axis = 1)

test_transformed = transformer.fit_transform(test_data[categorical_cols])
test_transformed_df = pd.DataFrame(test_transformed, columns=transformer.get_feature_names_out())
test_data = pd.concat([test_data, test_transformed_df], axis = 1)
test_data = test_data.drop(categorical_cols, axis = 1)

- onehot Encoding 를함으로써 문자형데이터를 수치형으로 변경햇다
- 아래는 종속변수를 제외한 데이터가 수치형으로 변한것을 볼수있다.

In [ ]:
train_data.info()

In [ ]:
test_data.info()

# Step 4. 머신러닝 모형 개발
- 모형에 대한 설명 필요
- 모형을 1-2개 사용
- 교차 검증
- 하이퍼파라미터 튜닝

- 독립변수와 종속변수를 구분해야 한다. 

In [ ]:
X_cols = test_data.columns
X = train_data[X_cols].to_numpy()
y = train_data['Transported'].to_numpy()

- 이번에는 훈련데이터를 검증데이터를 분리한다. 
- 이때 교차검증은 3회만 실시한다. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

- 이번에는 LightGBM 클래스를 부른 후 모형을 학습한다. 

In [ ]:

from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
lgb


# Model Selection 
- Ref. https://www.kaggle.com/code/samuelcortinhas/spaceship-titanic-a-complete-guide
- 그리드 서치를 적용함. 
- 4차시도 : 0.79097 (미 적용 시 : 0.78840)
    + 큰 차이는 일어나지 않음
    + 그러나 코드는 많이 작성...;;;

## 모형 옵션 선택
- 아래와 같은 모형을 정의한다. 

# Model Selection
- Logistic Regression는 선형 방정식을 사용한 분류 모형이다. 시그모이드 함수나 소프트맥스 함수를 사용하여 확률을 알 수 있다.
- K-NeighborsClassifier 는 맨해튼 거리 또는 유클라디안 거리를 사용해서 이웃의 개수가 많은 쪽으로 데이터를 분류하는 기법이다.
- RandomForest는 대표적인 의사결정트리 모형의 앙상블 학습 방법이다. 보통 회귀나 분류 문제를 위해 사용된다. 부트스트랩 샘플을 사용하고 랜덤하게 특성을 선택하여 트리를 만드는 방식을 사용한다.
- Light Gradient Boosting Machine(LGBM)은 XGBoost와 기본적으로 같고, 성능도 비슷하나, 더욱 빠르므로 선택했다. 참고로, XGBoost는 의사결정트리 모형의 앙상블로 구성된다.

In [ ]:
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

# Classifiers
classifiers = {
    "LogisticRegression" : LogisticRegression(random_state=0),
    "KNN" : KNeighborsClassifier(),
    "RandomForest" : RandomForestClassifier(random_state=0),
    "LGBM" : LGBMClassifier(random_state=0)
}

- 그리드 서치를 위한 옵션을 선택한다. 

In [ ]:
# Grids for grid search
LR_grid = {'penalty': ['l1','l2'],
           'C': [0.25, 0.5, 0.75, 1, 1.25, 1.5],
           'max_iter': [50, 100, 150]}

KNN_grid = {'n_neighbors': [3, 5, 7, 9],
            'p': [1, 2]}

RF_grid = {'n_estimators': [50, 100, 150, 200, 250, 300],
        'max_depth': [4, 6, 8, 10, 12]}

boosted_grid = {'n_estimators': [50, 100, 150, 200],
        'max_depth': [4, 8, 12],
        'learning_rate': [0.05, 0.1, 0.15]}

# Dictionary of all grids
grid = {
    "LogisticRegression" : LR_grid,
    "KNN" : KNN_grid,
    "RandomForest" : RF_grid,
    "LGBM" : boosted_grid
}

- 불러온 모형과 그리드 서치를 각각 개별적으로 정의하는 코드를 작성한다. 
- 해당 결과는 모두 데이터프레임에 담도록 한다. 

In [ ]:
# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
import time

i=0
clf_best_params=classifiers.copy()
valid_scores=pd.DataFrame({'Classifer':classifiers.keys(), 'Validation accuracy': np.zeros(len(classifiers)), 'Training time': np.zeros(len(classifiers))})
for key, classifier in classifiers.items():
    start = time.time()
    clf = GridSearchCV(estimator=classifier, param_grid=grid[key], n_jobs=-1, cv=None)

    # Train and score
    clf.fit(X_train, y_train)
    valid_scores.iloc[i,1]=clf.score(X_val, y_val)

    # Save trained model
    clf_best_params[key]=clf.best_params_
    
    # Print iteration and training time
    stop = time.time()
    valid_scores.iloc[i,2]=np.round((stop - start)/60, 2)
    
    print('Model:', key)
    print('Training time (mins):', valid_scores.iloc[i,2])
    print('')
    i+=1

- 4개의 모형에 대한 모형 개발 속도 및 정확도 평가지표를 확인한다. 

In [ ]:
# Show results
valid_scores

- 그리드 서치를통해 LGBM 과 RandomForest 의 점수가 가장 높은것으로 보여진다

In [ ]:
clf_best_params

In [ ]:
# Classifiers
best_classifiers = {
    "RandomForest" : RandomForestClassifier(**clf_best_params["RandomForest"], random_state=0),
    "LGBM" : LGBMClassifier(**clf_best_params["LGBM"], random_state=0)
}

- 이번에는 10겹 교차검증을 실시한다. 

In [ ]:
# Number of folds in cross validation
FOLDS=10

preds=np.zeros(len(test_data))
for key, classifier in best_classifiers.items():
    start = time.time()
    
    # 5-fold cross validation
    cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=0)
    
    score=0
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        # Get training and validation sets
        X_train, X_valid = X[train_idx], X[val_idx]
        y_train, y_valid = y[train_idx], y[val_idx]

        # Train model
        clf = classifier
        clf.fit(X_train, y_train)

        # Make predictions and measure accuracy
        preds += clf.predict_proba(test_data)[:,1]
        score += clf.score(X_valid, y_valid)

    # Average accuracy    
    score=score/FOLDS
    
    # Stop timer
    stop = time.time()

    # Print accuracy and time
    print('Model:', key)
    print('Average validation accuracy:', np.round(100*score,2))
    print('Training time (mins):', np.round((stop - start)/60,2))
    print('')
    
# Ensemble predictions
preds=preds/(FOLDS*len(best_classifiers))

# Step 5. 모형 평가
- 훈련데이터 쪼갠다. 훈련데이터 + 검증데이터 분리
- 정확도 비교
- 혼동행렬 (Confusion Matrix) 설명

# 혼동행렬 (Confusion Matrix)
- 주로 분류문제에서 모델의 성능을 평가할 때 혼동행렬 형태를 사용한다
- 4가지 개념 -> 앞이 T면 예측이 맞았음/ F이면 예측이 틀렸음을 의미 , 뒤가 P면 예측을 양성으로 했음/ N이면 예측을 음성으로 했음을 의미
    + TP/TN/FP/FN
    + TP는 true positive로 양성(1)으로 예측했는데 양성(1)일 경우를 말한다
    + TF는 true negative로 음성(0)으로 예측했는데 음성(0)일 경우를 말한다
    + FP는 false positive로 양성(1)으로 예측했는데 음성(0)일 경우를 말한다
    + FN는 false negative로 음성(0)으로 예측했는데 양성(1)일 경우를 말한다
- 평가지표 5가지
    + 정확도 = 전체에서 TP와 TF가 차지하는 비율 = (TP+TF) / (TP+TF+FP+FN)
    + 정밀도 = 양성으로 예측한 경우 중에서 실제 양성인 비율 = TP / (TP+FP)
    + 재현율 = 실제 양성 중에 제대로 예측한 비율 = TP / (TP+FN)
    + 특이도 = 실제 음성 중에 제대로 예측한 비율 = TN / (TN+FP)
    + F1 score = 정밀도와 재현율의 조화평균 = (2정밀도재현율) / (정밀도+재현율)
 + 참고자료 : https://taeguu.tistory.com/32

혼동행렬 오분류비용


- 먼저 cross_validate() 활용한다. 

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

splitter = StratifiedKFold(n_splits = 5, shuffle = True, random_state=42)
scores = cross_validate(lgb, X_train, y_train, return_train_score = True, cv=splitter)

print("train Acc.", np.mean(scores['train_score']))
print("test Acc.", np.mean(scores['test_score']))

- 이번에는 검증 데이터를 활용하여 정확도를 예상해본다. 

In [ ]:
from sklearn.metrics import accuracy_score

lgb.fit(X_train, y_train)
y_pred = lgb.predict(X_val)
print("Acc.", accuracy_score(y_val, y_pred))

# Step 6. 제출
- 제출 양식은 샘플 만들어드림

In [ ]:
# Round predictions to nearest integer
preds=np.round(preds).astype(bool)
sample_submission['Transported'] = preds
sample_submission.to_csv("submission.csv",index=False)
sample_submission.head()

- 혼동행렬, 분류성능평가지표

https://data-alpha.tistory.com/22

- 나무위키 : 혼동행렬

https://namu.wiki/w/%ED%98%BC%EB%8F%99%ED%96%89%EB%A0%AC